In [ ]:
import requests

# URL for the 24hr ticker price change statistics endpoint
url = "https://api.binance.com/api/v3/ticker/24hr"

# Make the request to get the ticker information
response = requests.get(url)
ticker_data = response.json()

# Print trading volume for each market
print("24hr Trading Volumes:")
for ticker in ticker_data:
    symbol = ticker['symbol']
    volume = ticker['volume']
    quote_volume = ticker['quoteVolume']
    print(f"Market: {symbol}, Volume: {volume}, Quote Volume: {quote_volume}")


In [ ]:
ticker_data


In [ ]:
import requests

# URL for the Binance exchange information endpoint
url = "https://api.binance.com/api/v3/exchangeInfo"

# Make the request to get the exchange info
response = requests.get(url)
exchange_info = response.json()

# Extract unique quote assets
quote_assets = set()
for symbol in exchange_info['symbols']:
    quote_assets.add(symbol['quoteAsset'])




In [ ]:
exchange_info

In [ ]:
import requests

# Define the URLs
exchange_info_url = "https://api.binance.com/api/v3/exchangeInfo"
ticker_24hr_url = "https://api.binance.com/api/v3/ticker/24hr"

# Send requests to both URLs
exchange_info_response = requests.get(exchange_info_url)
ticker_24hr_response = requests.get(ticker_24hr_url)

# Parse the JSON responses
exchange_info = exchange_info_response.json()
ticker_24hr_info = ticker_24hr_response.json()

# Create a dictionary to map symbols to their base and quote assets
symbol_info_map = {}
for symbol_info in exchange_info['symbols']:
    symbol = symbol_info['symbol']
    base_asset = symbol_info['baseAsset']
    quote_asset = symbol_info['quoteAsset']
    symbol_info_map[symbol] = {
        'symbol': symbol,
        'baseAsset': base_asset,
        'quoteAsset': quote_asset
    }

# Create the final list of dictionaries with the required keys
result = []
for ticker in ticker_24hr_info:
    symbol = ticker['symbol']
    if symbol in symbol_info_map:
        combined_info = symbol_info_map[symbol]
        combined_info['priceChangePercent'] = ticker['priceChangePercent']
        combined_info['quoteVolume'] = ticker['quoteVolume']
        result.append(combined_info)

# Print the result
print(result)


In [ ]:
import src.data_handler as m 

handler = m.DataHandler(min_volume=10e6)

result = handler.get_trading_pairs()

In [ ]:
for pair in result:
    print("------------------------------")
    print(pair)
    print("------------------------------")

print(len(result))

In [ ]:
for pair in result:
    print("------------------------------")
    print(pair)
    print("------------------------------")

print(len(result))

In [ ]:
import aiohttp
import asyncio

async def fetch(session, url):
    try:
        async with session.get(url) as response:
            if response.status == 200:
                return await response.json()
            else:
                return {'error': f"Failed to fetch {url}: {response.status}"}
    except aiohttp.ClientError as e:
        return {'error': f"Request failed for {url}: {str(e)}"}

async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        return await asyncio.gather(*tasks)

def process_responses(responses):
    responses_list = []
    for response in responses:
        if 'error' in response:
            print(response['error'])
        else:
            responses_list.append(response)
    return responses_list

urls = [
    'https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2024-05.zip',
    'https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2024-04.zip',
    'https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2024-03.zip',
    'https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2024-02.zip',
    'https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2024-01.zip'
]

async def main():
    responses = await fetch_all(urls)
    return process_responses(responses)

# Run the asyncio event loop
l = asyncio.run(main())

In [ ]:
import os

x = os.listdir('data/downloads')

unique = []

for file in x: 
    p = file.split('-')[0]
    unique.append(p)



In [ ]:
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import vectorbt as vbt

# Step 1: Read the Parquet file
df = pd.read_parquet('data/ETHUSDT/ETHUSDT-1m-2018-01.parquet')

# Ensure the column names are in lower case to match your data labels
df.columns = ['open_time', 'open', 'high', 'low', 'close']

# Set open_time as the DataFrame index
df['open_time'] = pd.to_datetime(df['open_time'])
df.set_index('open_time', inplace=True)

# Step 2: Calculate Ichimoku Cloud components
# Calculate Ichimoku Cloud components
ichimoku_cloud, _ = df.ta.ichimoku(tenkan_sen=True, kijun_sen=True, senkou_span=True, chikou_span=False)

# Extract individual components
tenkan_sen = ichimoku_cloud['ITS_9']
kijun_sen = ichimoku_cloud['IKS_26']
span_a = ichimoku_cloud['ISA_9']
span_b = ichimoku_cloud['ISB_26']

# Step 3: Define Strategy Logic
# Strategy rules: Long when close is above both span A and span B, and tenkan_sen is above kijun_sen
long_entries = (df['close'] > span_a) & (df['close'] > span_b) & (tenkan_sen > kijun_sen)
# Short when close is below both span A and span B, and tenkan_sen is below kijun_sen
short_entries = (df['close'] < span_a) & (df['close'] < span_b) & (tenkan_sen < kijun_sen)

# Step 4: Backtest the Strategy with vectorbt
# Create signals DataFrame
signals = pd.DataFrame(index=df.index)
signals['long'] = long_entries.astype(int)
signals['short'] = short_entries.astype(int)

# Create portfolio
portfolio = vbt.Portfolio.from_signals(
    close=df['close'],
    entries=signals['long'],
    exits=signals['short'],
    freq='1m'  # Daily frequency for closing positions
)

# Calculate portfolio statistics
stats = portfolio.stats()

# Step 5: Plotting the Backtest Results
# Plot the portfolio performance
portfolio.plot().show()

# Plot entries and exits on price chart
fig, ax = plt.subplots(figsize=(14, 7))
ax.plot(df.index, df['close'], label='Close')
ax.plot(tenkan_sen.index, tenkan_sen, label='Tenkan-sen', linestyle='--')
ax.plot(kijun_sen.index, kijun_sen, label='Kijun-sen', linestyle='--')
ax.plot(span_a.index, span_a, label='Span A', linestyle='--')
ax.plot(span_b.index, span_b, label='Span B', linestyle='--')

# Plot entry and exit signals
ax.plot(signals[signals['long'] == 1].index, df.loc[signals['long'] == 1, 'close'], '^', markersize=10, color='g', label='Long Entry')
ax.plot(signals[signals['short'] == 1].index, df.loc[signals['short'] == 1, 'close'], 'v', markersize=10, color='r', label='Short Entry')

ax.legend()
plt.title('Ichimoku Cloud Strategy')
plt.show()

# Print portfolio statistics
print(stats)


In [ ]:
df = pd.read_parquet('data/ETHUSDT/ETHUSDT-1m-2018-01.parquet')

# Ensure the column names are in lower case to match your data labels
df.columns = ['open_time', 'open', 'high', 'low', 'close']

# Set open_time as the DataFrame index
df['open_time'] = pd.to_datetime(df['open_time'])
df.set_index('open_time', inplace=True)

# Step 2: Calculate Ichimoku Cloud components
# Calculate Ichimoku Cloud components
ichimoku_cloud, _ = df.ta.ichimoku(tenkan_sen=True, kijun_sen=True, senkou_span=True, chikou_span=False)

In [ ]:
ichimoku_cloud

In [ ]:
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import vectorbt as vbt

# Step 1: Read the Parquet file
df = pd.read_parquet('data/ETHUSDT/ETHUSDT-1m-2018-01.parquet')

df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')

# Resample to 5-minute intervals

df_resampled = df.resample('1min', on='open_time').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last'
})


# Ensure all columns are properly aggregated
df = df_resampled.reset_index(inplace=False)


# Ensure the column names are in lower case to match your data labels
df.columns = ['open_time', 'open', 'high', 'low', 'close']

# Step 2: Calculate Ichimoku Cloud components
# Calculate Ichimoku Cloud components
ichimoku_cloud, _ = df.ta.ichimoku(tenkan_sen=True, kijun_sen=True, senkou_span=True, chikou_span=False)

# Extract individual components
tenkan_sen = ichimoku_cloud['ITS_9']
kijun_sen = ichimoku_cloud['IKS_26']
span_a = ichimoku_cloud['ISA_9']
span_b = ichimoku_cloud['ISB_26']

# Step 3: Define Strategy Logic
# Strategy rules: Long when close is above both span A and span B, and tenkan_sen is above kijun_sen
long_entries = (df['close'] > span_a) & (df['close'] > span_b) & (tenkan_sen > kijun_sen)
# Short when close is below both span A and span B, and tenkan_sen is below kijun_sen
short_entries = (df['close'] < span_a) & (df['close'] < span_b) & (tenkan_sen < kijun_sen)

# Exit conditions for long positions
long_exits = ~(tenkan_sen > kijun_sen) 

# Exit conditions for short positions
short_exits = ~(tenkan_sen < kijun_sen) 

# Step 4: Backtest the Strategy with vectorbt
# Create signals DataFrame
signals = pd.DataFrame(index=df.index)
signals['long'] = long_entries.astype(int)
signals['short'] = short_entries.astype(int)

# Create exit signals
signals['long_exit'] = long_exits.astype(int)
signals['short_exit'] = short_exits.astype(int)

# Create portfolio
portfolio = vbt.Portfolio.from_signals(
    close=df['close'],
    entries=signals['long'],
    exits=signals['long_exit'],
    short_entries=signals['short'],
    short_exits=signals['short_exit'],
    freq='1m'  # Daily frequency for closing positions
)

# Calculate portfolio statistics
stats = portfolio.stats()

# Step 5: Plotting the Backtest Results
# Plot the portfolio performance
portfolio.plot().show()

# Plot entries and exits on price chart
fig, ax = plt.subplots(figsize=(14, 7))
ax.plot(df.index, df['close'], label='Close')
ax.plot(tenkan_sen.index, tenkan_sen, label='Tenkan-sen', linestyle='--')
ax.plot(kijun_sen.index, kijun_sen, label='Kijun-sen', linestyle='--')
ax.plot(span_a.index, span_a, label='Span A', linestyle='--')
ax.plot(span_b.index, span_b, label='Span B', linestyle='--')

# Plot entry signals
ax.plot(signals[signals['long'] == 1].index, df.loc[signals['long'] == 1, 'close'], '^', markersize=10, color='g', label='Long Entry')
ax.plot(signals[signals['short'] == 1].index, df.loc[signals['short'] == 1, 'close'], 'v', markersize=10, color='r', label='Short Entry')

# Plot exit signals
ax.plot(signals[signals['long_exit'] == 1].index, df.loc[signals['long_exit'] == 1, 'close'], 'o', markersize=7, color='g', label='Long Exit')
ax.plot(signals[signals['short_exit'] == 1].index, df.loc[signals['short_exit'] == 1, 'close'], 'o', markersize=7, color='r', label='Short Exit')

ax.legend()
plt.title('Ichimoku Cloud Strategy with Exit Signals')
plt.show()

# Print portfolio statistics
print(stats)


In [ ]:
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import vectorbt as vbt

# Step 1: Read the Parquet file
df = pd.read_parquet('data/ETHUSDT/ETHUSDT-1m-2023-09.parquet')

# Convert 'open_time' to datetime assuming it's in milliseconds
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')

# Resample to 5-minute intervals
df_resampled = df.resample('1min', on='open_time').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last'
})

# Ensure all columns are properly aggregated
df = df_resampled.reset_index()

# Ensure the column names are in lower case to match your data labels
df.columns = ['open_time', 'open', 'high', 'low', 'close']

# Step 2: Calculate Ichimoku Cloud components
# Calculate Ichimoku Cloud components
ichimoku_cloud, _ = df.ta.ichimoku(tenkan_sen=True, kijun_sen=True, senkou_span=True, chikou_span=False)

# Extract individual components
tenkan_sen = ichimoku_cloud['ITS_9']
kijun_sen = ichimoku_cloud['IKS_26']
span_a = ichimoku_cloud['ISA_9']
span_b = ichimoku_cloud['ISB_26']

# Step 3: Define Strategy Logic

# Strategy rules: Long when close is above both span A and span B, tenkan_sen is above kijun_sen, and RSI < 30
long_entries = (df['close'] > span_a) & (df['close'] > span_b) & (tenkan_sen > kijun_sen) 
# Short when close is below both span A and span B, tenkan_sen is below kijun_sen, and RSI > 70
short_entries = (df['close'] < span_a) & (df['close'] < span_b) & (tenkan_sen < kijun_sen)

# Exit conditions for long positions: tenkan_sen < kijun_sen or RSI > 70
long_exits = (tenkan_sen < kijun_sen) 
# Exit conditions for short positions: tenkan_sen > kijun_sen or RSI < 30
short_exits = (tenkan_sen > kijun_sen) 

# Step 4: Backtest the Strategy with vectorbt
# Create signals DataFrame
signals = pd.DataFrame(index=df.index)
signals['long'] = long_entries.astype(int)
signals['short'] = short_entries.astype(int)

# Create exit signals
signals['long_exit'] = long_exits.astype(int)
signals['short_exit'] = short_exits.astype(int)

# Create portfolio
portfolio = vbt.Portfolio.from_signals(
    close=df['close'],
    entries=signals['long'],
    exits=signals['long_exit'],
    short_entries=signals['short'],
    short_exits=signals['short_exit'],
    freq='1m'  # 5-minute frequency for closing positions
)

# Calculate portfolio statistics
stats = portfolio.stats()

# Step 5: Plotting the Backtest Results
# Plot the portfolio performance
portfolio.plot().show()

# Plot entries and exits on price chart
fig, ax = plt.subplots(figsize=(14, 7))
ax.plot(df['open_time'], df['close'], label='Close')
ax.plot(tenkan_sen.index, tenkan_sen, label='Tenkan-sen', linestyle='--')
ax.plot(kijun_sen.index, kijun_sen, label='Kijun-sen', linestyle='--')
ax.plot(span_a.index, span_a, label='Span A', linestyle='--')
ax.plot(span_b.index, span_b, label='Span B', linestyle='--')

# Plot entry signals
ax.plot(signals[signals['long'] == 1].index, df.loc[signals['long'] == 1, 'close'], '^', markersize=10, color='g', label='Long Entry')
ax.plot(signals[signals['short'] == 1].index, df.loc[signals['short'] == 1, 'close'], 'v', markersize=10, color='r', label='Short Entry')

# Plot exit signals
ax.plot(signals[signals['long_exit'] == 1].index, df.loc[signals['long_exit'] == 1, 'close'], 'o', markersize=7, color='g', label='Long Exit')
ax.plot(signals[signals['short_exit'] == 1].index, df.loc[signals['short_exit'] == 1, 'close'], 'o', markersize=7, color='r', label='Short Exit')

ax.legend()
plt.title('Ichimoku Cloud Strategy with RSI Conditions')
plt.show()

# Print portfolio statistics
print(stats)


In [ ]:
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import vectorbt as vbt
import os 


def backtest(data):
    # Step 1: Read the Parquet file
    df = data

    # Convert 'open_time' to datetime assuming it's in milliseconds
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    
    # Resample to 5-minute intervals
    df_resampled = df.resample('1min', on='open_time').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })
    
    # Ensure all columns are properly aggregated
    df = df_resampled.reset_index()
    
    # Ensure the column names are in lower case to match your data labels
    df.columns = ['open_time', 'open', 'high', 'low', 'close']
    
    # Step 2: Calculate Ichimoku Cloud components
    # Calculate Ichimoku Cloud components
    ichimoku_cloud, _ = df.ta.ichimoku(tenkan_sen=True, kijun_sen=True, senkou_span=True, chikou_span=False)
    
    # Extract individual components
    tenkan_sen = ichimoku_cloud['ITS_9']
    kijun_sen = ichimoku_cloud['IKS_26']
    span_a = ichimoku_cloud['ISA_9']
    span_b = ichimoku_cloud['ISB_26']
    
    # Step 3: Define Strategy Logic
    # Calculate RSI
    df['rsi'] = ta.rsi(df['close'], length=14)
    
    # Define Strategy Logic
    # Long when close is above both span A and span B, tenkan_sen is above kijun_sen, and RSI < 30
    long_entries = (df['close'] > span_a) & (df['close'] > span_b) & (tenkan_sen > kijun_sen)
    # Short when close is below both span A and span B, tenkan_sen is below kijun_sen, and RSI > 70
    short_entries = (df['close'] < span_a) & (df['close'] < span_b) & (tenkan_sen < kijun_sen)
    
    # Exit conditions for long positions: tenkan_sen < kijun_sen or RSI > 70
    long_exits = (tenkan_sen < kijun_sen) | (df['rsi'] > 70)
    # Exit conditions for short positions: tenkan_sen > kijun_sen or RSI < 30
    short_exits = (tenkan_sen > kijun_sen) | (df['rsi'] < 30) 
    
    # Step 4: Backtest the Strategy with vectorbt
    # Create signals DataFrame
    signals = pd.DataFrame(index=df.index)
    signals['long'] = long_entries.astype(int)
    signals['short'] = short_entries.astype(int)
    
    # Create exit signals
    signals['long_exit'] = long_exits.astype(int)
    signals['short_exit'] = short_exits.astype(int)
    
    # Create portfolio
    portfolio = vbt.Portfolio.from_signals(
        close=df['close'],
        entries=signals['long'],
        exits=signals['long_exit'],
        short_entries=signals['short'],
        short_exits=signals['short_exit'],
        freq='1m',  # 5-minute frequency for closing positions
    )
    
    # Calculate portfolio statistics
    stats = portfolio.stats()
    
    # Step 5: Plotting the Backtest Results
    # Plot the portfolio performance
    portfolio.plot().show()
    # Print portfolio statistics
    print(stats)

def concat_parquet_files(file_paths):
    dfs = []
    
    # Iterate through each file path
    for file_path in file_paths:
        # Read the parquet file
        df = pd.read_parquet(file_path)
        
        # Convert 'open_time' to datetime assuming it's in milliseconds
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        
        # Append the DataFrame to the list
        dfs.append(df)
    
    # Concatenate all DataFrames in the list along rows
    concatenated_df = pd.concat(dfs, ignore_index=True)
    
    return concatenated_df

count=0
paths = []
for file in os.listdir('data/DOGEUSDT'):
    if file.endswith('.parquet'):
        paths.append(os.path.join('data/DOGEUSDT', file))


eth_data = concat_parquet_files(paths)





In [ ]:
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import vectorbt as vbt
import os 
import src.utils as ut

def backtest(file):
    # Step 1: Read the Parquet file
    df = pd.read_parquet(file)

    # Convert 'open_time' to datetime assuming it's in milliseconds
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    
    # Resample to 1-minute intervals (adjust if needed)
    df_resampled = df.resample('15min', on='open_time').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })
    
    # Ensure all columns are properly aggregated
    df = df_resampled.reset_index()
    
    # Ensure the column names are in lower case to match your data labels
    df.columns = ['open_time', 'open', 'high', 'low', 'close']
    
    # Step 2: Calculate Ichimoku Cloud components
    # Calculate Ichimoku Cloud components
    ichimoku_cloud, _ = df.ta.ichimoku(tenkan_sen=True, kijun_sen=True, senkou_span=True, chikou_span=False)
    
    # Extract individual components
    tenkan_sen = ichimoku_cloud['ITS_9']
    kijun_sen = ichimoku_cloud['IKS_26']
    span_a = ichimoku_cloud['ISA_9']
    span_b = ichimoku_cloud['ISB_26']
    
    # Step 3: Calculate RSI
    df['rsi'] = ta.rsi(df['close'], length=14)
    
    # Step 4: Define Strategy Logic
    # Create signals DataFrame
    signals = pd.DataFrame(index=df.index)
    
    # Define Strategy Logic
    # Long when tenkan_sen crosses over kijun_sen, close is above both span A and span B, and RSI < 30
    long_entries = (ut.crossed_over(tenkan_sen, kijun_sen)) & (df['close'] > span_a) & (df['close'] > span_b) 
    
    # Short when tenkan_sen crosses below kijun_sen, close is below both span A and span B, and RSI > 70
    short_entries = (ut.crossed_below(tenkan_sen, kijun_sen)) & (df['close'] < span_a) & (df['close'] < span_b) 
    
    # Exit conditions for long positions: tenkan_sen < kijun_sen or RSI > 70
    long_exits = ut.crossed_below(tenkan_sen, kijun_sen)
    # Exit conditions for short positions: tenkan_sen > kijun_sen or RSI < 30
    short_exits = ut.crossed_over(tenkan_sen, kijun_sen) 
    
    # Assign strategy signals to the signals DataFrame
    signals['long'] = long_entries.astype(int)
    signals['short'] = short_entries.astype(int)
    signals['long_exit'] = long_exits.astype(int)
    signals['short_exit'] = short_exits.astype(int)
    
    # Step 5: Backtest the Strategy with vectorbt
    # Create portfolio
    portfolio = vbt.Portfolio.from_signals(
        close=df['close'],
        entries=signals['long'],
        exits=signals['long_exit'],
        short_entries=signals['short'],
        short_exits=signals['short_exit'],
        freq='1min',  # Frequency for closing positions (adjust as needed)
        fees=0.001
    )
    
    # Calculate portfolio statistics
    stats = portfolio.stats()
    
    # Step 6: Plotting the Backtest Results
    # Plot the portfolio performance
    portfolio.plot().show()
    # Print portfolio statistics
    print(stats)




def backtest_data(data):
    # Step 1: Read the Parquet file
    df = data

    # Convert 'open_time' to datetime assuming it's in milliseconds
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    
    # Resample to 1-minute intervals (adjust if needed)
    df_resampled = df.resample('15min', on='open_time').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })
    
    # Ensure all columns are properly aggregated
    df = df_resampled.reset_index()
    
    # Ensure the column names are in lower case to match your data labels
    df.columns = ['open_time', 'open', 'high', 'low', 'close']
    
    # Step 2: Calculate Ichimoku Cloud components
    # Calculate Ichimoku Cloud components
    ichimoku_cloud, _ = df.ta.ichimoku(tenkan_sen=True, kijun_sen=True, senkou_span=True, chikou_span=False)
    
    # Extract individual components
    tenkan_sen = ichimoku_cloud['ITS_9']
    kijun_sen = ichimoku_cloud['IKS_26']
    span_a = ichimoku_cloud['ISA_9']
    span_b = ichimoku_cloud['ISB_26']
    
    # Step 3: Calculate RSI
    df['rsi'] = ta.rsi(df['close'], length=14)
    
    # Step 4: Define Strategy Logic
    # Create signals DataFrame
    signals = pd.DataFrame(index=df.index)
    
    # Define Strategy Logic
    # Long when tenkan_sen crosses over kijun_sen, close is above both span A and span B, and RSI < 30
    long_entries = (ut.crossed_over(tenkan_sen, kijun_sen)) & (df['close'] > span_a) & (df['close'] > span_b) 
    
    # Short when tenkan_sen crosses below kijun_sen, close is below both span A and span B, and RSI > 70
    short_entries = (ut.crossed_below(tenkan_sen, kijun_sen)) & (df['close'] < span_a) & (df['close'] < span_b) 
    
    # Exit conditions for long positions: tenkan_sen < kijun_sen or RSI > 70
    long_exits = ut.crossed_below(tenkan_sen, kijun_sen)
    # Exit conditions for short positions: tenkan_sen > kijun_sen or RSI < 30
    short_exits = ut.crossed_over(tenkan_sen, kijun_sen) 
    
    # Assign strategy signals to the signals DataFrame
    signals['long'] = long_entries.astype(int)
    signals['short'] = short_entries.astype(int)
    signals['long_exit'] = long_exits.astype(int)
    signals['short_exit'] = short_exits.astype(int)
    
    # Step 5: Backtest the Strategy with vectorbt
    # Create portfolio
    portfolio = vbt.Portfolio.from_signals(
        close=df['close'],
        entries=signals['long'],
        exits=signals['long_exit'],
        short_entries=signals['short'],
        short_exits=signals['short_exit'],
        freq='1min',  # Frequency for closing positions (adjust as needed)
        fees=0.001
    )
    
    # Calculate portfolio statistics
    stats = portfolio.stats()
    
    # Step 6: Plotting the Backtest Results
    # Plot the portfolio performance
    portfolio.plot().show()
    # Print portfolio statistics
    print(stats)

def concat_parquet_files(file_paths):
    dfs = []
    
    # Iterate through each file path
    for file_path in file_paths:
        # Read the parquet file
        df = pd.read_parquet(file_path)
        
        # Convert 'open_time' to datetime assuming it's in milliseconds
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        
        # Append the DataFrame to the list
        dfs.append(df)
    
    # Concatenate all DataFrames in the list along rows
    concatenated_df = pd.concat(dfs, ignore_index=True)
    
    return concatenated_df

paths = []
for file in os.listdir('data/DOGEUSDT'):
    if file.endswith('.parquet'):
        paths.append(os.path.join('data/DOGEUSDT', file))

d = concat_parquet_files(paths)

backtest_data(d)



In [ ]:
##############################
####### OPTIMIZATION #########
##############################

import pandas as pd 
import pandas_ta as ta 
import vectorbt as vbt
import numpy as np
from src.utils import crossed_below, crossed_over
import os
import time


def backtest_strategy(data, tenkan_sen_length, kijun_sen_length, senkou_span_length):
    # Step 1: Read the Parquet file
    df = data.copy()

    # Convert 'open_time' to datetime assuming it's in milliseconds
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    
    # Resample to 15-minute intervals (adjust if needed)
    df_resampled = df.resample('15min', on='open_time').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })
    
    # Ensure all columns are properly aggregated
    df = df_resampled.reset_index()
    
    # Ensure the column names are in lower case to match your data labels
    df.columns = ['open_time', 'open', 'high', 'low', 'close']
    
    # Step 2: Calculate Ichimoku Cloud components
    # Calculate Ichimoku Cloud components with specified parameters
    ichimoku_cloud, _ = ta.ichimoku(high= df['high'], low=df['low'], close=df['close'], tenkan=tenkan_sen_length, kijun=kijun_sen_length, senkou=senkou_span_length, include_chikou=False)

    # ichimoku_cloud.rename(columns={'ISA_9': f'ISA_{senkou_span_length//2}',
    #                                 'ISB_26':f'ISB_{senkou_span_length}',
    #                                 'ITS_9':f'ITS_{tenkan_sen_length}',
    #                                 'IKS_26':f'IKS_{kijun_sen_length}' }, inplace=True)
    # ichimoku_cloud.drop(columns=['ICS_26'])
    
    # Extract individual components
    tenkan_sen = ichimoku_cloud[f'ITS_{tenkan_sen_length}']
    kijun_sen = ichimoku_cloud[f'IKS_{kijun_sen_length}']
    span_a = ichimoku_cloud[f'ISA_{tenkan_sen_length}']  # Adjust for span A
    span_b = ichimoku_cloud[f'ISB_{kijun_sen_length}']
    
    # Step 3: Define Strategy Logic
    # Create signals DataFrame
    signals = pd.DataFrame(index=df.index)
    
    # Define Strategy Logic based on extracted components
    # Long when tenkan_sen crosses over kijun_sen, close is above both span A and span B
    long_entries = crossed_over(tenkan_sen, kijun_sen) & (df['close'] > span_a) & (df['close'] > span_b)
    
    # Short when tenkan_sen crosses below kijun_sen, close is below both span A and span B
    short_entries = crossed_below(tenkan_sen, kijun_sen) & (df['close'] < span_a) & (df['close'] < span_b) 
    
    # Exit conditions for long positions: tenkan_sen < kijun_sen
    long_exits = crossed_below(tenkan_sen, kijun_sen)
    
    # Exit conditions for short positions: tenkan_sen > kijun_sen
    short_exits = crossed_over(tenkan_sen, kijun_sen)
    
    # Assign strategy signals to the signals DataFrame
    signals['long'] = long_entries.astype(int)
    signals['short'] = short_entries.astype(int)
    signals['long_exit'] = long_exits.astype(int)
    signals['short_exit'] = short_exits.astype(int)
    
    # Step 4: Backtest the Strategy with vectorbt
    # Create portfolio
    portfolio = vbt.Portfolio.from_signals(
        close=df['close'],
        entries=signals['long'],
        exits=signals['long_exit'],
        short_entries=signals['short'],
        short_exits=signals['short_exit'],
        freq='15min',  # Frequency for closing positions (adjust as needed)
        fees=0.001
    )
    
    # Calculate portfolio statistics
    stats = portfolio.stats()
    
    return stats

def optimize_strategy(data):
    parameter_grid = {
        'tenkan_sen_length': range(5, 51, 5),    # Adjust range and step size as needed
        'kijun_sen_length': range(20, 61, 5),
        'senkou_span_length': range(30, 91, 5)
    }
    
    results = {}
    
    for tenkan_sen_length in parameter_grid['tenkan_sen_length']:
        for kijun_sen_length in parameter_grid['kijun_sen_length']:
            for senkou_span_length in parameter_grid['senkou_span_length']:
                # Run backtest with current parameter combination
                print(f"Testing Params: {tenkan_sen_length} ,{kijun_sen_length} ,{senkou_span_length}")
                stats = backtest_strategy(data, tenkan_sen_length, kijun_sen_length, senkou_span_length)
                
                # Store results
                results[(tenkan_sen_length, kijun_sen_length, senkou_span_length)] = stats
    
    return results


def find_best_parameters(results, metric='Sharpe Ratio'):
    # Convert results dictionary to DataFrame for easier manipulation
    results_df = pd.DataFrame.from_dict(results, orient='index')
    
    # Find parameters that maximize the chosen metric
    if metric == 'Sharpe Ratio':
        best_params = results_df['Sharpe Ratio'].idxmax()
    elif metric == 'Total Return':
        best_params = results_df['Total Return'].idxmax()
    # Add more metrics as needed
    
    best_stats = results_df.loc[best_params]
    
    return best_params, best_stats

import plotly.graph_objects as go

def plot_scatter(results, metric='Sharpe Ratio'):
    # Extract parameter values and metric values from results
    tenkan_sen_values = [key[0] for key in results.keys()]
    kijun_sen_values = [key[1] for key in results.keys()]
    senkou_span_values = [key[2] for key in results.keys()]
    
    # Extract metric values
    metric_values = []
    for key in results.keys():
        stats = results[key]
        if metric == 'Sharpe Ratio':
            metric_value = stats['Sharpe Ratio']
        elif metric == 'Total Return':
            metric_value = stats['Total Return']
        else:
            metric_value = None  # Handle additional metrics if needed
        metric_values.append(metric_value)
    
    # Create scatter plot using Plotly
    fig = go.Figure(data=go.Scatter3d(
        x=tenkan_sen_values,
        y=kijun_sen_values,
        z=senkou_span_values,
        mode='markers',
        marker=dict(
            size=12,
            color=metric_values,
            colorscale='Viridis',  # Choose a colorscale
            opacity=0.8,
            colorbar=dict(title=metric)
        )
    ))
    
    fig.update_layout(
        scene=dict(
            xaxis_title='Tenkan-sen Length',
            yaxis_title='Kijun-sen Length',
            zaxis_title='Senkou Span Length'
        ),
        title=f'Optimization Scatter Plot ({metric})'
    )
    
    fig.show()

def concat_parquet_files(file_paths):
    dfs = []
    
    # Iterate through each file path
    for file_path in file_paths:
        # Read the parquet file
        df = pd.read_parquet(file_path)
        
        # Convert 'open_time' to datetime assuming it's in milliseconds
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        
        # Append the DataFrame to the list
        dfs.append(df)
    
    # Concatenate all DataFrames in the list along rows
    concatenated_df = pd.concat(dfs, ignore_index=True)
    
    return concatenated_df

print("reading data...")
paths = []
for file in os.listdir('data/ETHUSDT'):
    if file.endswith('.parquet'):
        paths.append(os.path.join('data/ETHUSDT', file))

d = concat_parquet_files(paths)
print('optimizing strategy ><><><')
start = time.time()
results = optimize_strategy(d.tail(int(len(d)/40)))
print('done optimization')
best_params, best_stats = find_best_parameters(results, metric='Sharpe Ratio')

print("Best Parameters:", best_params)
print("Best Stats:")
print(best_stats)



print(f'total time is: {time.time()-sy}')



In [ ]:
import pandas as pd
import pandas_ta as ta
import vectorbt as vbt
from src.utils import crossed_below, crossed_over
import os
import time
import multiprocessing
import plotly.graph_objects as go
import numpy as np

def backtest_strategy(data, tenkan_sen_length, kijun_sen_length, senkou_span_length):
    # Step 1: Read the Parquet file
    df = data.copy()

    # Convert 'open_time' to datetime assuming it's in milliseconds
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')

    # # Resample to 15-minute intervals (adjust if needed)
    # df_resampled = df.resample('1min', on='open_time').agg({
    #     'open': 'first',
    #     'high': 'max',
    #     'low': 'min',
    #     'close': 'last'
    # })

    # # Ensure all columns are properly aggregated
    # df = df_resampled.reset_index()

    # Ensure the column names are in lower case to match your data labels
    df.columns = ['open_time', 'open', 'high', 'low', 'close']

    # Step 2: Calculate Ichimoku Cloud components
    # Calculate Ichimoku Cloud components with specified parameters
    ichimoku_cloud, _ = ta.ichimoku(high=df['high'], low=df['low'], close=df['close'],
                                    tenkan=tenkan_sen_length, kijun=kijun_sen_length, senkou=senkou_span_length,
                                    include_chikou=False)

    # Extract individual components
    tenkan_sen = ichimoku_cloud[f'ITS_{tenkan_sen_length}']
    kijun_sen = ichimoku_cloud[f'IKS_{kijun_sen_length}']
    span_a = ichimoku_cloud[f'ISA_{tenkan_sen_length}']  # Adjust for span A
    span_b = ichimoku_cloud[f'ISB_{kijun_sen_length}']

    # Step 3: Define Strategy Logic
    # Create signals DataFrame
    signals = pd.DataFrame(index=df.index)

    # Define Strategy Logic based on extracted components
    # Long when tenkan_sen crosses over kijun_sen, close is above both span A and span B
    long_entries = crossed_over(tenkan_sen, kijun_sen) & (df['close'] > span_a) & (df['close'] > span_b)

    # Short when tenkan_sen crosses below kijun_sen, close is below both span A and span B
    short_entries = crossed_below(tenkan_sen, kijun_sen) & (df['close'] < span_a) & (df['close'] < span_b)

    # Exit conditions for long positions: tenkan_sen < kijun_sen
    long_exits = crossed_below(tenkan_sen, kijun_sen)

    # Exit conditions for short positions: tenkan_sen > kijun_sen
    short_exits = crossed_over(tenkan_sen, kijun_sen)

    # Assign strategy signals to the signals DataFrame
    signals['long'] = long_entries.astype(int)
    signals['short'] = short_entries.astype(int)
    signals['long_exit'] = long_exits.astype(int)
    signals['short_exit'] = short_exits.astype(int)

    # Step 4: Backtest the Strategy with vectorbt
    # Create portfolio
    portfolio = vbt.Portfolio.from_signals(
        close=df['close'],
        high=df['high'],
        low=df['low'],
        entries=signals['long'],
        exits=signals['long_exit'],
        short_entries=signals['short'],
        short_exits=signals['short_exit'],
        freq='1m',  # Frequency for closing positions (adjust as needed)
        fees=0.001
    )

    # Calculate portfolio statistics
    stats = portfolio.stats()

    return stats

def optimize_strategy_worker(params):
    data, tenkan_sen_length, kijun_sen_length, senkou_span_length = params
    print()
    return (tenkan_sen_length, kijun_sen_length, senkou_span_length), backtest_strategy(data, tenkan_sen_length, kijun_sen_length, senkou_span_length)

def optimize_strategy(data):
    parameter_grid = {
        'tenkan_sen_length': range(5, 51, 10),    # Adjust range and step size as needed
        'kijun_sen_length': range(20, 61, 10),
        'senkou_span_length': range(30, 91, 10)
    }

    results = {}
    params_list = [(data, t, k, s) for t in parameter_grid['tenkan_sen_length']
                   for k in parameter_grid['kijun_sen_length']
                   for s in parameter_grid['senkou_span_length']]

    # Use multiprocessing Pool to parallelize
    with multiprocessing.Pool() as pool:
        for params, result in pool.imap_unordered(optimize_strategy_worker, params_list):
            results[params] = result

    return results

def plot_scatter(results, metric='Sharpe Ratio'):
    # Extract parameter values and metric values from results
    tenkan_sen_values = [key[0] for key in results.keys()]
    kijun_sen_values = [key[1] for key in results.keys()]
    senkou_span_values = [key[2] for key in results.keys()]
    
    # Extract metric values
    metric_values = []
    for key in results.keys():
        stats = results[key]
        if metric == 'Sharpe Ratio':
            metric_value = stats['Sharpe Ratio']
        elif metric == 'Total Return':
            metric_value = stats['Total Return']
        else:
            metric_value = None  # Handle additional metrics if needed
        metric_values.append(metric_value)
    
    # Create scatter plot using Plotly
    fig = go.Figure(data=go.Scatter3d(
        x=tenkan_sen_values,
        y=kijun_sen_values,
        z=senkou_span_values,
        mode='markers',
        marker=dict(
            size=12,
            color=metric_values,
            colorscale='Viridis',  # Choose a colorscale
            opacity=0.8,
            colorbar=dict(title=metric)
        )
    ))
    
    fig.update_layout(
        scene=dict(
            xaxis_title='Tenkan-sen Length',
            yaxis_title='Kijun-sen Length',
            zaxis_title='Senkou Span Length'
        ),
        title=f'Optimization Scatter Plot ({metric})'
    )
    
    fig.show()

def plot_surface_with_contours(results, metric='Sharpe Ratio'):
    # Extract parameter values and metric values from results
    parameter_combinations = list(results.keys())
    tenkan_sen_values = sorted(set([params[0] for params in parameter_combinations]))
    kijun_sen_values = sorted(set([params[1] for params in parameter_combinations]))
    senkou_span_values = [params[2] for params in parameter_combinations]
    
    # Extract metric values
    metric_values = []
    for params in parameter_combinations:
        stats = results[params]
        if metric == 'Sharpe Ratio':
            metric_value = stats['Sharpe Ratio']
        elif metric == 'Total Return [%]':
            metric_value = stats.get('Total Return [%]', None)  # Use get() to handle missing keys gracefully
        else:
            metric_value = None  # Handle additional metrics if needed
        metric_values.append(metric_value)
    
    # Create meshgrid for plotting
    X, Y = np.meshgrid(tenkan_sen_values, kijun_sen_values)
    
    # Ensure Z is initialized with float type for NaNs
    Z = np.zeros_like(X, dtype=np.float64)
    
    # Populate Z with metric values
    for i, params in enumerate(parameter_combinations):
        tenkan_index = tenkan_sen_values.index(params[0])
        kijun_index = kijun_sen_values.index(params[1])
        if metric_values[i] is not None:
            Z[kijun_index, tenkan_index] = metric_values[i]
    
    # Create Surface plot using Plotly
    fig = go.Figure(data=[go.Surface(
        x=X,
        y=Y,
        z=Z,
        contours_z=dict(
            show=True,
            usecolormap=True,
            highlightcolor="limegreen",
            project_z=True,
            highlightwidth=4,
        ),
        colorscale='Viridis'  # Choose a colorscale
    )])
    
    fig.update_layout(
        scene=dict(
            xaxis_title='Tenkan-sen Length',
            yaxis_title='Kijun-sen Length',
            zaxis_title='Metric Value' if metric else 'Senkou Span Length'
        ),
        title=f'Optimization Surface Plot ({metric})',
        autosize=True,
        width=800,
        height=600,
        margin=dict(l=65, r=50, b=65, t=90)
    )
    
    fig.show()

def concat_parquet_files(file_paths):
    dfs = []

    # Iterate through each file path
    for file_path in file_paths:
        # Read the parquet file
        df = pd.read_parquet(file_path)

        # Convert 'open_time' to datetime assuming it's in milliseconds
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')

        # Append the DataFrame to the list
        dfs.append(df)

    # Concatenate all DataFrames in the list along rows
    concatenated_df = pd.concat(dfs, ignore_index=True)

    return concatenated_df

if __name__ == "__main__":
    print("Reading data...")
    paths = [os.path.join('data/ETHUSDT', file) for file in os.listdir('data/ETHUSDT') if file.endswith('.parquet')]
    data = concat_parquet_files(paths)

    print('Optimizing strategy...')
    start = time.time()
    results = optimize_strategy(data.tail(int(len(data)/50)))
    print('Done optimization')

    best_params, best_stats = max(results.items(), key=lambda x: x[1]['Total Return [%]'])

    print('\n')
    print("---------------------")
    print("Best Parameters:", best_params)
    print("Best Stats:")
    print(best_stats)

    print(f'Total time taken: {time.time() - start} seconds')
    print('\n')
    print("---------------------")
    plot_surface_with_contours(results, metric='Total Return')



In [ ]:
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
import os 

def chandelier_exit_signals(df, length=22, mult=3.0, use_close=True):
    df_copy = df.copy()
    
    # Calculate ATR
    df_copy['atr'] = ta.atr(df_copy['high'], df_copy['low'], df_copy['close'], length)

    # Calculate Long Stop and Short Stop
    if use_close:
        df_copy['long_stop'] = df_copy['close'].rolling(length).max() - mult * df_copy['atr']
        df_copy['short_stop'] = df_copy['close'].rolling(length).min() + mult * df_copy['atr']
    else:
        df_copy['long_stop'] = df_copy['high'].rolling(length).max() - mult * df_copy['atr']
        df_copy['short_stop'] = df_copy['low'].rolling(length).min() + mult * df_copy['atr']

    # Forward fill initial stop values
    df_copy['long_stop'] = df_copy['long_stop'].ffill()
    df_copy['short_stop'] = df_copy['short_stop'].ffill()

    # Calculate directional logic
    df_copy['dir'] = 1
    df_copy.loc[df_copy['close'] < df_copy['long_stop'].shift(1), 'dir'] = -1
    df_copy.loc[df_copy['close'] > df_copy['short_stop'].shift(1), 'dir'] = 1

    # Generate buy and sell signals
    df_copy['buy_signal'] = (df_copy['dir'] == 1) & (df_copy['dir'].shift(1) == -1)
    df_copy['confirmed_buy_signal'] = df_copy['buy_signal'] & df_copy['dir'].shift(1).notna()

    return df_copy['confirmed_buy_signal']

def calculate_zlsma(df, length=50, offset=0, src='close'):
    df_copy = df.copy()
    src_series = df_copy[src]

    # Calculate the first LSMA
    df_copy['lsma'] = ta.linreg(src_series, length, offset)

    # Calculate the second LSMA using the first LSMA
    df_copy['lsma2'] = ta.linreg(df_copy['lsma'], length, offset)

    # Calculate the Zero Lag LSMA
    df_copy['eq'] = df_copy['lsma'] - df_copy['lsma2']
    df_copy['zlsma'] = df_copy['lsma'] + df_copy['eq']

    return df_copy['zlsma']

def heikin_ashi(df):
    column_names = df.columns.tolist()
    heikin_ashi_df = ta.ha(open_=df['open'], high=df['high'], low=df['low'], close=df['close'])
    heikin_ashi_df.columns = column_names[1:]
    return heikin_ashi_df

def entry_exit_signals(df, chandelier_signals, zlsma):
    df_copy = df.copy()
    
    # Entry signal: confirmed buy signal and current price is above ZLSMA
    df_copy['entry_signal'] = chandelier_signals & (df_copy['close'] > zlsma)

    # Exit signal: price closes below ZLSMA
    df_copy['exit_signal'] = False  # Initialize exit signals column

    # Track the state of being in a position
    in_position = False

    for i in range(1, len(df_copy)):
        if df_copy.at[i, 'entry_signal']:  # If entry signal found
            in_position = True  # Set position to true
        elif in_position and (df_copy.at[i, 'close'] < zlsma.at[i]):  # If in position and exit condition
            df_copy.at[i, 'exit_signal'] = True  # Mark exit signal
            in_position = False  # Reset position

    return df_copy[['entry_signal', 'exit_signal']]


def plot_signals(df, chandelier_signals, zlsma, entry_exit_signals):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                         open=df['open'],
                                         high=df['high'],
                                         low=df['low'],
                                         close=df['close'],
                                         name='Candlesticks')])

    # Add Chandelier Exit buy signals
    buy_signals = df[chandelier_signals]
    fig.add_trace(go.Scatter(x=buy_signals.index,
                             y=buy_signals['close'],
                             mode='markers',
                             marker=dict(color='green', size=10, symbol='triangle-up'),
                             name='Chandelier Buy Signal'))

    # Add ZLSMA
    fig.add_trace(go.Scatter(x=df.index,
                             y=zlsma,
                             mode='lines',
                             line=dict(color='white', width=2),
                             name='Zero Lag LSMA'))

    # Add entry signals
    entry_signals = df[entry_exit_signals['entry_signal']]
    fig.add_trace(go.Scatter(x=entry_signals.index,
                             y=entry_signals['close'],
                             mode='markers',
                             marker=dict(color='blue', size=10, symbol='circle'),
                             name='Entry Signal'))

    # Add exit signals
    exit_signals = df[entry_exit_signals['exit_signal']]
    fig.add_trace(go.Scatter(x=exit_signals.index,
                             y=exit_signals['close'],
                             mode='markers',
                             marker=dict(color='red', size=10, symbol='x'),
                             name='Exit Signal'))

    # Update layout
    fig.update_layout(title='Candlestick Chart with Buy/Sell Signals',
                      yaxis_title='Price',
                      xaxis_title='Date')

    fig.show()

def create_portfolio(heikin_ashi_df, entry_exit_df, initial_cash=10000, slippage=0.001, fees=0.001):
    # Initialize portfolio
    portfolio = vbt.Portfolio.from_signals(
        close=heikin_ashi_df['close'],
        entries=entry_exit_df['entry_signal'],
        exits=entry_exit_df['exit_signal'],
        size=1.0,  # Always invest 100% of capital
        direction='longonly',  # Only go long
        fees=fees,  # Transaction fees in percent
        slippage=slippage,  # Slippage in percent
        freq='5min'  # Assuming frequency of trading signals
    )

    return portfolio

def concat_parquet_files(file_paths):
    dfs = []

    # Iterate through each file path
    for file_path in file_paths:
        # Read the parquet file
        df = pd.read_parquet(file_path)

        # Convert 'open_time' to datetime assuming it's in milliseconds
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')

        # Append the DataFrame to the list
        dfs.append(df)

    # Concatenate all DataFrames in the list along rows
    concatenated_df = (pd.concat(dfs, ignore_index=True)).sort_values(by='open_time', ascending=True)
    concatenated_df.reset_index(drop=True, inplace=True)
    return concatenated_df

path = 'data/BTCUSDT'
print("Reading data...")
paths = [os.path.join(path, file) for file in os.listdir(path) if file.endswith('.parquet')]
data = concat_parquet_files(paths)
df_resampled = data.resample('5min', on='open_time').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })
heikin_ashi_df = heikin_ashi(df_resampled)
chandelier_signals = chandelier_exit_signals(heikin_ashi_df)
zlsma = calculate_zlsma(heikin_ashi_df)
entry_exit_df = entry_exit_signals(heikin_ashi_df, chandelier_signals, zlsma)
print("Backtesting...")

portfolio = create_portfolio(heikin_ashi_df, entry_exit_df)

print(portfolio.stats())

portfolio.plot().show()

